# Root Exceptions

When building a module’s API it’s also very important to consider the `exceptions` the module will throw. Python comes with built in exceptions but there are drawbacks to using the built-in exception types. Consider the example below where a `ValueError` is raised when an invalid parameter is passed to a function:

In [1]:
def convert_to_celsius(kelvin: int) -> float:
    if kelvin < 0:
        raise ValueError('Kelvin must be >= 0')

In some cases, `ValueError` is the most appropriate error, but when building an API it’s much more powerful to build your own hierarchy of exceptions. This can be achieved by providing a root `Exception` in your module. Once this is defined, have all your other exceptions inherit from the root exception:

In [2]:
class RootError(Exception):
    'Base class for all exceptions'
    
class InvalidKelvinError(RootError):
    'The value of kelvin is invalid'

Having a root exception makes it easier for consumers of your API to catch errors thrown intentionally. This also helps callers understand when there’s a problem with their usage of the API. 

When this is used in a `try/except` block it prevents your API’s exceptions propagating too far upward and breaking the calling program. Hence, insulating the calling code from your API:

In [ ]:
try:
    celsius = some_module.convert_to_celsius(-50)
except some_module.InvalidKelvinError:
    celsius = some_module.convert_to_celsius(0)
except some_module.RootError as e:
    print(f'Potential bug in calling code: {e}') # should really be using logger!

If your code deliberately raises exceptions as defined by your module, then all other exceptions raised by your module must ones not intended to be raised.  These are potential bugs in your API’s code.

The above `try/except` will not insulate API consumers from bugs in your APIs module’s code. To do this, the caller will need to add another `except` block that will catch Python’s base `Exception` class. This allows the consumer of the API to detect bugs in the API’s module:

In [ ]:
try:
    celsius = some_module.convert_to_celsius(-50)
except some_module.InvalidKelvinError:
    celsius = some_module.convert_to_celsius(0)
except some_module.RootError as e:
    print(f'Potential bug in calling code: {e}') # should really be using logger!
except Exception as e:
    print(f'Potential bug in API code: {e}') # Again, should be using logger!
    raise